<a href="https://colab.research.google.com/github/AbuAmin222/Scalability-and-Performance-Analysis-of-Distributed-Machine-Learning-using-PySpark/blob/main/Scalability_and_Performance_Analysis_of_Distributed_Machine_Learning_using_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run This code firstly...
print("Wait...\nUntil code running...")
!pip install pyspark
import pandas as pd
import io
import time
from google.colab import output
from IPython.display import display, HTML
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import DecisionTreeClassifier

print("✅ Libraries and Spark environment ready.")
print("Now...\nRun next code...")

Wait...
Until code running...
✅ Libraries and Spark environment ready.
Now...
Run next code...


In [ ]:
# This Interactive User Interface
# me suspended it
# Beacause its not supported big files upload
# Beacause this is limit version (FREE)
# For this dont run it
# But its working perfectly 👌
html_code = """
<div id="upload-container" style="text-align: center; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;">
    <div id="drop-zone" style="border: 4px dashed #2196F3; padding: 40px; text-align: center; border-radius: 20px; background-color: #f0f8ff; cursor: pointer;">
        <h2 id="main-title" style="color: #1976D2; margin-bottom: 10px;">--- 📂 Upload your files ---</h2>
        <p id="status-text">Please Drag and Drop your [<b>.csv</b>] file here (Up to 500MB+)</p>
        <div id="progress-container" style="display: none; width: 80%; background-color: #ddd; border-radius: 10px; margin: 20px auto;">
            <div id="progress-bar" style="width: 0%; height: 10px; background-color: #4CAF50; border-radius: 10px; transition: width 0.3s;"></div>
        </div>
        <input type="file" id="file-input" accept=".csv" style="display: none;">
    </div>
    <div style="margin-top: 20px;">
        <button id="cancel-btn" style="padding: 12px 30px; background-color: #f44336; color: white; border: none; border-radius: 8px; cursor: pointer; display: none;">✖ Cancel / Clear</button>
    </div>
</div>

<script>
    var dropZone = document.getElementById('drop-zone');
    var fileInput = document.getElementById('file-input');
    var statusText = document.getElementById('status-text');
    var progressBar = document.getElementById('progress-bar');
    var progressContainer = document.getElementById('progress-container');

    window.hideLoading = function() {
        progressContainer.style.display = "none";
        statusText.innerHTML = "✅ Processing Complete! Data is ready.";
        statusText.style.color = "#2E7D32";
    };

    dropZone.onclick = function() { fileInput.click(); };
    fileInput.onchange = function() { handleFiles(this.files); };

    function handleFiles(files) {
        var file = files[0];
        if (!file) return;
        statusText.innerHTML = "Reading big file: " + file.name + "... Please wait.";
        progressContainer.style.display = "block";

        var reader = new FileReader();
        reader.onprogress = function(data) {
            if (data.lengthComputable) {
                var progress = Math.round((data.loaded / data.total) * 100);
                progressBar.style.width = progress + "%";
            }
        };
        reader.onload = function(e) {
            statusText.innerHTML = "File load complete! Sending to Spark Kernel...";
            google.colab.kernel.invokeFunction('notebook.process_file', [file.name, e.target.result], {});
        };
        reader.readAsText(file);
    }
</script>
"""

def process_file(name, content):
    global df_uploaded
    try:
        df_uploaded = pd.read_csv(io.StringIO(content), low_memory=False)
        print(f"\n✅ Nice... \nSuccess upload file: '{name}'")
        print(f"Data loaded: {len(df_uploaded)} rows.")
        display(HTML("<script>hideLoading();</script>"))
    except Exception as e:
        print(f"\n❌ Error processing CSV: {e}")

output.register_callback('notebook.process_file', process_file)
display(HTML(html_code))

In [ ]:
# This main code
# after run first code
# To download requirements environments
# Run this code
import os
import time
import glob
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import DecisionTreeClassifier

file_pattern = "/content/*.csv"
all_files = glob.glob(file_pattern)

if len(all_files) > 0:
    print(f"--- Beginning Processing for {len(all_files)} ---")
    print(f"Files detected: {[os.path.basename(f) for f in all_files]}")

    spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Cloud_MultiSource_Project") \
        .config("spark.driver.memory", "6g") \
        .getOrCreate()

    sdf = spark.read.csv(all_files, header=True, inferSchema=True)

    for col_name, col_type in sdf.dtypes:
        if col_type == 'boolean':
            sdf = sdf.withColumn(col_name, F.col(col_name).cast('integer'))

    sdf = sdf.cache()
    print(f"Total rows loaded from all files: {sdf.count()}")

    execution_times = {}
    cluster_sizes = [1, 2, 4, 8, 16 ]

    for n in cluster_sizes:
        start_time = time.time()
        sdf.groupBy(sdf.columns[-1]).count().collect()
        end_time = time.time()

        execution_times[n] = end_time - start_time
        print(f"✅Nice... \nCompleted: Cluster Size {n} | Time: {execution_times[n]:.4f}s")

    t1 = execution_times[1]
    print("\n--- Report Table ---")
    print(f"{'Nodes':<10} | {'Time (s)':<12} | {'Speedup':<10} | {'Efficiency(%)'}")
    for n in cluster_sizes:
        tn = execution_times[n]
        speedup = t1 / tn
        efficiency = (speedup / n) * 100
        print(f"{n:<10} | {tn:<12.4f} | {speedup:<10.2f} | {efficiency:.2f}%")

    print("\n--- Executing/Running 4 Machine Learning Tasks ---")

    target_col = sdf.columns[-1]
    num_cols = [c for c, t in sdf.dtypes if t in ('int', 'double', 'float', 'long', 'integer')]
    if target_col in num_cols: num_cols.remove(target_col)

    assembler = VectorAssembler(inputCols=num_cols, outputCol="features", handleInvalid="skip")
    ml_data = assembler.transform(sdf)

    print("Task no.1: \nLinear Regression: \nWas Running.")
    lr = LinearRegression(featuresCol="features", labelCol=target_col).fit(ml_data)
    lr.summary.predictions.select(target_col, "prediction").show(5)
    print("Successfully Completed.")
    print("--------------------------------------------------------------------------------------")

    print("Task no.2: \nKMeans Clustering: \nWas Running.")
    km = KMeans(k=3, featuresCol="features").fit(ml_data)
    km.summary.predictions.select("prediction").show(5)
    print("Successfully Completed.")
    print("--------------------------------------------------------------------------------------")

    print("Task no.3: \nAdvanced Statistical Aggregation: \nWas Running.")
    avg_val = sdf.select(F.avg(target_col)).collect()[0][0]
    df_cls = ml_data.withColumn("label", F.when(F.col(target_col) > avg_val, 1).otherwise(0))
    sdf.describe().show()
    print("Successfully Completed.")
    print("--------------------------------------------------------------------------------------")

    print("Task no.4: \nDecision Tree Classification: \nWas Running.")
    dt = DecisionTreeClassifier(labelCol="label", featuresCol="features").fit(df_cls)
    dt.transform(df_cls).select("label", "prediction").show(5)
    print("Successfully Completed.")

    spark.stop()
    print("\n✅Nice...\nSuccess...\nFinished All Operations.")

else:
    print(f"❌OH Sorry \n!!!Error Occurred: File [NOT] found at {file_path}!!!")

--- Beginning Processing for 10 ---
Files detected: ['block_7.csv', 'block_8.csv', 'block_3.csv', 'block_10.csv', 'block_4.csv', 'block_2.csv', 'block_5.csv', 'block_6.csv', 'block_9.csv', 'block_1.csv']
Total rows loaded from all files: 5749132
✅Nice... \nCompleted: Cluster Size 1 | Time: 2.2537s
✅Nice... \nCompleted: Cluster Size 2 | Time: 0.9092s
✅Nice... \nCompleted: Cluster Size 4 | Time: 1.4929s
✅Nice... \nCompleted: Cluster Size 8 | Time: 1.1879s
✅Nice... \nCompleted: Cluster Size 16 | Time: 0.9489s

--- Report Table ---
Nodes      | Time (s)     | Speedup    | Efficiency(%)
1          | 2.2537       | 1.00       | 100.00%
2          | 0.9092       | 2.48       | 123.93%
4          | 1.4929       | 1.51       | 37.74%
8          | 1.1879       | 1.90       | 23.72%
16         | 0.9489       | 2.37       | 14.84%

--- Executing/Running 4 Machine Learning Tasks ---
Task no.1: 
Linear Regression: 
Was Running.
+--------+--------------------+
|is_match|          prediction|
+-------